In [42]:
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re, string
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
fname = 'tweets.txt'

In [43]:
lines = [line.rstrip() for line in open(fname)]
file = open("newfile.txt", "w")

In [44]:
def checkTopic (topic, text):
    if (set(topic) & set(text)):
        return True
    return False

def bag_of_words(words):
    if len(words) > 0:
        return dict([(word, True) for word in words])

stop = stopwords.words('english')

In [45]:
json_input = lines[4]
decoded = json.loads(json_input)
    
# pretty printing of json-formatted string
print json.dumps(decoded, sort_keys=True, indent=4)

nf_timestamp = decoded['timestamp_ms']
print "nf_timestamp", nf_timestamp

nf_user = decoded['user']['screen_name']
print "nf_user", nf_user

{
    "contributors": null, 
    "coordinates": null, 
    "created_at": "Sun Jun 12 18:07:18 +0000 2016", 
    "entities": {
        "hashtags": [], 
        "symbols": [], 
        "urls": [], 
        "user_mentions": []
    }, 
    "favorite_count": 0, 
    "favorited": false, 
    "filter_level": "low", 
    "geo": null, 
    "id": 742055693281288200, 
    "id_str": "742055693281288193", 
    "in_reply_to_screen_name": null, 
    "in_reply_to_status_id": null, 
    "in_reply_to_status_id_str": null, 
    "in_reply_to_user_id": null, 
    "in_reply_to_user_id_str": null, 
    "is_quote_status": false, 
    "lang": "en", 
    "place": null, 
    "retweet_count": 0, 
    "retweeted": false, 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "text": "Thread of my fav mutuals \ud83d\udc96", 
    "timestamp_ms": "1465754838663", 
    "truncated": false, 
    "user": {
        "contributors_enabled": false, 
        "created_at":

In [71]:
total = 0
exception = 0
topic = ["terrorism"]
print topic
corpus = []
label = []
count = 0
counts = []
words = []
sentences = []

for index in range(len(lines)):
    json_input = lines[index]
    decoded = json.loads(json_input)
    
    try:

        nf_lang = decoded['user']['lang']
        #print "nf_lang", nf_lang
    
        if nf_lang == 'en':
            #print "JSON parsing example: ", decoded['hashtags']
            # cal treure entities and stop words
            nf_tweet_text = decoded['text']
            #print "nf_tweet_text", nf_tweet_text
            nf_RT = (decoded['retweet_count']) > 0
            #print "nf_RT", nf_RT
            if nf_RT:
                nf_rt_text = decoded['retweeted_status']['text']
                nf_tweet_text = nf_rt_text
            #print "nf_tweet_text :: ", nf_tweet_text

            # not alphabetical characters
            nf_tweet_text = re.sub(r"[^a-zA-Z]", " ", nf_tweet_text)
            
            s = nf_tweet_text.split()          
            #print(s)
 
            # filtered = [word for words in s if word not in stopwords.words('english')]
            nf_tweet_filtered_text = [i for i in s if i not in stop]
            #print "nf_tweet_filtered_text :: ", nf_tweet_filtered_text
            nf_tweet_filtered_text = [element.lower() for element in nf_tweet_filtered_text]
            
            # check the topic
            aux = " ".join( nf_tweet_filtered_text )
            corpus.append(aux)
            count = count + 1
            nf_positiu = checkTopic(topic, nf_tweet_filtered_text)
            if nf_positiu:
                total = total + 1
                label.append(1)
                sentences.append(aux)
                print index, count, total, "**", aux
            else:
                label.append(0)
            counts.append(len(nf_tweet_text)) 
            words.append(len(nf_tweet_filtered_text))

               
            nf_words = []
        
            nf_count_hashtags = len(decoded['entities']['hashtags'])
            #print "nf_count_hastags", nf_count_hashtags
            nf_pos_first_hashtag = 0
            nf_tweet_hashtags = []
            if nf_count_hashtags > 0:
                nf_pos_first_hashtag = decoded['entities']['hashtags'][0]['indices'][0]
                #print "nf_pos_first_hashtag", nf_pos_first_hashtag
                for i in range(nf_count_hashtags):
                    nf_tweet_hashtags.append(decoded['entities']['hashtags'][i]['text'])
                #print "nf_tweet_hashtags", nf_tweet_hashtags
                nf_tweet_hashtags = [element.lower() for element in nf_tweet_hashtags]
                nf_words.append(nf_tweet_hashtags)
        
            nf_count_urls = len(decoded['entities']['urls'])
            #print "nf_count_urls", nf_count_urls
            nf_pos_first_url = 0
            nf_tweet_urls = []
            if nf_count_urls > 0:
                nf_pos_first_url = decoded['entities']['urls'][0]['indices'][0]
                #print "nf_pos_first_url", nf_pos_first_url
                for i in range (nf_count_urls):
                    nf_tweet_urls.append(decoded['entities']['urls'][i]['url'])
                #print nf_tweet_urls
                #nf_tweet_urls = [element.lower() for element in nf_tweet_urls]
                #nf_words.append(nf_tweet_urls)
        
            nf_count_mentions = len(decoded['entities']['user_mentions'])
            #print "nf_count_mentions", nf_count_mentions
            nf_pos_first_mention = 0
            nf_tweet_mentions = []
            if nf_count_mentions > 0:
                nf_pos_first_mention = decoded['entities']['user_mentions'][0]['indices'][0]
                #print "nf_pos_first_mention", nf_pos_first_mention    
                for i in range(nf_count_mentions):
                    nf_tweet_mentions.append(decoded['entities']['user_mentions'][i]['screen_name']) 
                #print "nf_tweet_mentions", nf_tweet_mentions
                nf_tweet_mentions = [element.lower() for element in nf_tweet_mentions]
                nf_words.append(nf_tweet_mentions) 

            num_list = [0]
            if nf_pos_first_hashtag > 0:
                num_list.append(nf_pos_first_hashtag)
            if nf_pos_first_url > 0:
                num_list.append(nf_pos_first_url)
            if nf_pos_first_mention > 0:
                num_list.append(nf_pos_first_mention)
            nf_pos_first_entity = min(num_list)
            #print "nf_pos_first_entity", nf_pos_first_entity

            nf_user = decoded['user']['screen_name']
            #print "nf_user", nf_user

            nf_timestamp = decoded['timestamp_ms']
            #print "nf_timestamp", nf_timestamp
            
            #file.write(nf_tweet_filtered_text, nf_positiu, nf_count_hashtags, nf_pos_first_hashtag)


    except (KeyError):
        print "JSON format error", KeyError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
    except (ValueError):
        print "JSON format error", ValueError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
    except (TypeError):
        print "JSON format error", TypeError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
        

print total, count, index, exception
print max(counts), min(counts), sum(counts)/len(counts)
print max(words), min(words), sum(words)/len(words)
#print corpus
print sentences

['terrorism']
12 9 1 ** perezhilton heartbreaking that i teach kids day one love respect everyone this still terrorism towards gays
109 79 2 ** rt realdonaldtrump is president obama going finally mention words radical islamic terrorism if immediately r
138 100 3 ** rt nero this terrorism it islamism it extremism this islam
282 189 4 ** rt realdonaldtrump is president obama going finally mention words radical islamic terrorism if immediately r
612 432 5 ** rt andreagrimes i fear officials calling terrorism
830 586 6 ** rt thegopreport looks like jeff sessions right attacks we need trump lead us amp strong terrorism http
919 655 7 ** potus realdonaldtrump hillaryclinton radical islamic terrorism hate wow blinded ideology made us unsafe fact
7 710 999 0
156 7 95
28 1 11
[u'perezhilton heartbreaking that i teach kids day one love respect everyone this still terrorism towards gays', u'rt realdonaldtrump is president obama going finally mention words radical islamic terrorism if immediately 

In [72]:
file.close()

In [73]:
#vectorizer = CountVectorizer(min_df=1, stop_words='english')
vectorizer = CountVectorizer(stop_words='english')
vectorizer

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [74]:
X = vectorizer.fit_transform(sentences)
X

<7x49 sparse matrix of type '<type 'numpy.int64'>'
	with 72 stored elements in Compressed Sparse Row format>

In [75]:
print (vectorizer.vocabulary_)
vocab = vectorizer.get_feature_names()
print vocab
print len(vocab)

{u'right': 38, u'love': 26, u'perezhilton': 32, u'jeff': 21, u'radical': 35, u'unsafe': 46, u'realdonaldtrump': 36, u'officials': 31, u'looks': 25, u'nero': 29, u'immediately': 17, u'need': 28, u'respect': 37, u'fear': 8, u'hate': 12, u'blinded': 3, u'islam': 18, u'rt': 39, u'thegopreport': 44, u'lead': 23, u'wow': 48, u'finally': 9, u'going': 11, u'islamism': 20, u'terrorism': 43, u'hillaryclinton': 14, u'http': 15, u'sessions': 40, u'extremism': 6, u'andreagrimes': 1, u'islamic': 19, u'heartbreaking': 13, u'words': 47, u'president': 34, u'teach': 42, u'strong': 41, u'day': 5, u'obama': 30, u'kids': 22, u'like': 24, u'amp': 0, u'mention': 27, u'attacks': 2, u'calling': 4, u'ideology': 16, u'trump': 45, u'gays': 10, u'potus': 33, u'fact': 7}
[u'amp', u'andreagrimes', u'attacks', u'blinded', u'calling', u'day', u'extremism', u'fact', u'fear', u'finally', u'gays', u'going', u'hate', u'heartbreaking', u'hillaryclinton', u'http', u'ideology', u'immediately', u'islam', u'islamic', u'islamis

In [76]:
print X.toarray()

[[0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0
  1 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1
  0 0 1 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1
  0 0 1 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 1 0 0 0 1 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0
  0 1 1 1 1 0 1 1 1 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
  0 0 0 0 0 0 1 0 0 1 0 1]]


In [77]:
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print tfidf.fit_transform(vectorizer.fit_transform(sentences)).toarray()

[[ 0.    0.    0.    0.    0.    0.35  0.    0.    0.    0.    0.35  0.    0.
   0.35  0.    0.    0.    0.    0.    0.    0.    0.    0.35  0.    0.    0.
   0.35  0.    0.    0.    0.    0.    0.35  0.    0.    0.    0.    0.35
   0.    0.    0.    0.    0.35  0.15  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.32  0.    0.32
   0.    0.    0.    0.    0.    0.32  0.    0.27  0.    0.    0.    0.    0.
   0.    0.    0.32  0.    0.    0.32  0.    0.    0.    0.32  0.27  0.27
   0.    0.    0.21  0.    0.    0.    0.16  0.    0.    0.    0.32  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.47  0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.47  0.    0.47  0.    0.    0.    0.    0.
   0.    0.    0.    0.47  0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.26  0.    0.    0.    0.2   0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.32  0.    0.32
   0.    0.    0.    0.    0.

In [78]:
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(corpus)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
# the word topic should be the most present
for tag, count in zip(vocab, dist):
    print count, tag

1 amp
2 andreagrimes
1 attacks
1 blinded
4 calling
1 day
1 extremism
1 fact
1 fear
2 finally
6 gays
2 going
1 hate
2 heartbreaking
1 hillaryclinton
1 http
1 ideology
1 immediately
1 islam
1 islamic
1 islamism
8 jeff
2 kids
1 lead
3 like
1 looks
1 love
1 mention
4 need
1 nero
1 obama
3 officials
1 perezhilton
1 potus
2 president
1 radical
1 realdonaldtrump
1 respect
1 right
1 rt
1 sessions
2 strong
4 teach
3 terrorism
1 thegopreport
1 trump
1 unsafe
1 words
1 wow


In [79]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
arraylabel = np.asarray(label)
print arraylabel
print len(arraylabel)
print len(corpus)


Training the random forest...
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [80]:
forest = forest.fit( train_data_features, np.array(label))
#predicted= model.predict(x_test)

In [99]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

test = sentences[0]
print test
usersentences = sentences[1:]
print sentences


# Set values for various parameters
num_features = 300   # Word vector dimensionality                      
min_word_count = 1   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(usersentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

print model

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)
print "fi"
#model.accuracy('/tmp/questions-words.txt')
model.doesnt_match(test)

perezhilton heartbreaking that i teach kids day one love respect everyone this still terrorism towards gays
[u'perezhilton heartbreaking that i teach kids day one love respect everyone this still terrorism towards gays', u'rt realdonaldtrump is president obama going finally mention words radical islamic terrorism if immediately r', u'rt nero this terrorism it islamism it extremism this islam', u'rt realdonaldtrump is president obama going finally mention words radical islamic terrorism if immediately r', u'rt andreagrimes i fear officials calling terrorism', u'rt thegopreport looks like jeff sessions right attacks we need trump lead us amp strong terrorism http', u'potus realdonaldtrump hillaryclinton radical islamic terrorism hate wow blinded ideology made us unsafe fact']
Training model...
Word2Vec(vocab=24, size=300, alpha=0.025)
fi


u'm'

In [97]:
bigram_transformer = gensim.models.Phrases(sentences)

In [98]:
print bigram_transformer 

Phrases<212 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [100]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print(basemodel)

basemodel.build_vocab(sentences)  

Word2Vec(vocab=0, size=100, alpha=0.025)
